In [1]:
import sys
sys.path.append('python-chess')
import chess
import chess.uci
import random
import time
import cProfile
import numpy as np

In [26]:
def simpleHeuristic(state, result = None, role = None):
    d = {1: 1, 2: 3, 3: 3, 4: 5, 5: 9}
    heuristicScore = 0
    for i in range(1, 6):
        heuristicScore += d[i] * (len(state.pieces(i, True)) - len(state.pieces(i, False)))
    if (state.is_check()):
        heuristicScore += .5
    if not state.turn:
        heuristicScore = -heuristicScore
    return heuristicScore

def reward(state, result, role):
    if role:
        return -100 if result == "0-1" else -100
    else:
        return 100 if result == "1-0" else -1000
    
def monteCarloHeuristic(state, depth, role):
    result = state.is_game_over(True)
    if result != "*":
        return reward(state, result, role)
    elif not depth:
        return simpleHeuristic(state)
    randomMove = randomMove(state)
    state.push(randomMove)
    score = monteCarloHeuristic(state,depth-1,role)
    state.pop()
    return score

class player():
    def __init__(self, depth = 1, heuristic = simpleHeuristic, verbose = False):
        self.depth = depth
        self.heuristic = heuristic
        self.verbose = verbose

def gameOver(state):
    return state.result(True)

def randomMove(state):
    moves = state.legal_moves
    numMoves = len(moves)
    moveNumber = random.randint(0, numMoves-1)
    for i, move in enumerate(moves):
        if i == moveNumber:
            return move
        
class randomPlayer(player):
    def getMove(self, state):
        return randomMove(state)

class minimaxPlayer(player):
    def maxScore(self,state, depth):
        result = gameOver(state)
        if result != "*":
            return (None, reward(state, result, state.turn))
        if not depth:
            return (None, self.heuristic(state))
        moves = state.legal_moves
        bestScore = float("-inf")
        for i, move in enumerate(moves):
            state.push(move)
            testScore = self.minScore(state, depth)
            if testScore > bestScore:
                bestMove = move
                bestScore = testScore
                if bestScore == 100:
                    state.pop()
                    break
            state.pop()
            
        return (bestMove, bestScore)
            
            
    def minScore(self, state, depth):
        result = gameOver(state)
        if result != "*":
            return reward(state, result, state.turn)
            
        moves = state.legal_moves
        worstScore = float("inf")
        for i, move in enumerate(moves):
            state.push(move)
            testMove, testScore = self.maxScore(state, depth - 1)
            if testScore < worstScore:
                worstScore = testScore
                if worstScore == -100:
                    state.pop()
                    break
            state.pop()
        return worstScore
    
    def getMove(self, state):
        bestMove, bestScore = self.maxScore(state, self.depth)
        return bestMove
    
class alphabetaPlayer(player):
    def maxScore(self,state, alpha, beta, depth):
        result = gameOver(state)
        if result != "*":
            return (None, reward(state, result, state.turn))
        if not depth:
            return (None, self.heuristic(state, 100, state.turn))
        moves = state.legal_moves
        bestMove = None
        for i, move in enumerate(moves):
            state.push(move)
            testScore = self.minScore(state, alpha, beta, depth)
            if testScore > alpha:
                bestMove = move
                alpha = testScore
                if alpha >= beta:
                    state.pop()
                    break
            state.pop()
            
        return (bestMove, alpha)
            
            
    def minScore(self, state, alpha, beta, depth):
        result = gameOver(state)
        if result != "*":
            return reward(state, result, state.turn)
            
        moves = state.legal_moves
        for i, move in enumerate(moves):
            state.push(move)
            testMove, testScore = self.maxScore(state, alpha, beta, depth - 1)
            if testScore < beta:
                beta = testScore
                if beta <= alpha:
                    state.pop()
                    break
            state.pop()
        return beta
    
    def getMove(self, state):
        bestMove, bestScore = self.maxScore(state, -100, 100, self.depth)
        return bestMove

In [18]:
def play(state, p1, p2, verbose = False):
    while True:
        if verbose:
            print()
            print(state)
            print()
        
        result = gameOver(state)
        if (result != "*"):
            return result

        whiteMove = p1.getMove(state)

        state.push(whiteMove)
        
        if verbose:
            print()
            print(state)
            print()
        
        result = gameOver(state)
        if (result != "*"):
            return result

        blackMove = p2.getMove(state)

        state.push(blackMove)
        
def evaluate(p1, p2, n, verbose=False):
    state = chess.Board()
    white = 0
    black = 0
    for i in range(n):
        result = play(state, p1 , p2, verbose)
        if result == "1-0":
            white += 1
        elif result == "0-1":
            black += 1
        else:
            white += .5
            black += .5
        state.reset()
    print(white,black)

In [27]:
#Random vs Random
p1 = randomPlayer(depth=0, heuristic=None,verbose=False)
p2 = randomPlayer(depth=0, heuristic=None,verbose=False)

evaluate(p1,p2,30)

16.0 14.0


In [153]:
#Minimax vs Random
p1 = minimaxPlayer(depth=1, heuristic=simpleHeuristic, verbose=False)
p2 = randomPlayer(depth=0, heuristic=None, verbose=False)
evaluate(p1,p2,1)

1 0


In [36]:
#alphabeta vs Random
p1 = alphabetaPlayer(depth=2, heuristic=simpleHeuristic, verbose=False)
p2 = randomPlayer(depth=0, heuristic=None, verbose=False)
evaluate(p1,p2,1, True)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R


r n b q k b n r
p p p p p p p .
. . . . . . . .
. . . . . . . p
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R


r n b q k b n r
p p p p p p p .
. . . . . . . .
. . . . . . N p
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R


r n b q k b n r
p . p p p p p .
. . . . . . . .
. p . . . . N p
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R


r n b q k b n r
p . p p p p p .
. . . . . . . .
. p . . . . N p
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B R .


r n b q k b n r
p . . p p p p .
. . . . . . . .
. p p . . . N p
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B R .


r n b q k b n r
p . . p p p p .
. . . . . . . .
. p p . . . . p
. . . . N . . .
. . . . .

In [137]:
board = chess.Board()

True


In [31]:
#alphabeta w/ MCS vs Random
p1 = alphabetaPlayer(depth=2, heuristic=monteCarloHeuristic, verbose=False)
p2 = randomPlayer(depth=0, heuristic=None, verbose=False)
evaluate(p1,p2,1, True)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R



AttributeError: 'NoneType' object has no attribute 'from_square'

In [35]:
#alphabeta vs alphabeta
p1 = alphabetaPlayer(depth=2, heuristic=simpleHeuristic, verbose=False)
p2 = alphabetaPlayer(depth=2, heuristic=simpleHeuristic, verbose=False)
evaluate(p1,p2,1, True)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R


r n b q k b n r
p p p p p p p .
. . . . . . . p
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R


r n b q k b n r
p p p p p p p .
. . . . . . . p
. . . . . . . .
. . . . . N . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R


r n b q k b . r
p p p p p p p .
. . . . . n . p
. . . . . . . .
. . . . . N . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R


r n b q k b . r
p p p p p p p .
. . . . . n . p
. . . . . . . .
. . . . . N . .
. . . . . P . .
P P P P P . P P
R N B Q K B . R


r n b q k b r .
p p p p p p p .
. . . . . n . p
. . . . . . . .
. . . . . N . .
. . . . . P . .
P P P P P . P P
R N B Q K B . R


r n b q k b r .
p p p p p p p .
. . . . . n . p
. . . . . . . .
. . . . . . . .
. . . . .


r . b q k r . .
p p . p . p . .
n . p b p n p p
. . . . . . . .
. . . . . . . .
. P . K P P . P
P B P P . N P .
R N . Q . B . R


r . b q k r . .
p p . p . p . .
n . p b p n p p
. . . . . . . .
. . K . . . . .
. P . . P P . P
P B P P . N P .
R N . Q . B . R


r . b q k . r .
p p . p . p . .
n . p b p n p p
. . . . . . . .
. . K . . . . .
. P . . P P . P
P B P P . N P .
R N . Q . B . R


r . b q k . r .
p p . p . p . .
n . p b p n p p
. . . . . . . .
. . K . . . . .
. P . N P P . P
P B P P . . P .
R N . Q . B . R


r . b q k r . .
p p . p . p . .
n . p b p n p p
. . . . . . . .
. . K . . . . .
. P . N P P . P
P B P P . . P .
R N . Q . B . R


r . b q k r . .
p p . p . p . .
n . p b p n p p
. . . . . . . .
. . K . . . . .
. P . . P P . P
P B P P . N P .
R N . Q . B . R


r . b . k r . .
p p . p q p . .
n . p b p n p p
. . . . . . . .
. . K . . . . .
. P . . P P . P
P B P P . N P .
R N . Q . B . R


r . b . k r . .
p p . p q p . .
n . p b p n p p
. . . . . . . .
P . K . . . . .
. P . . P

KeyboardInterrupt: 